### Solution to Problem Statement 

In [ ]:
import csv
import numpy as np
import math

# For clustering of stocks into sectors
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform

# For making neural net models of indices
import torch
from torch import nn
from torch import optim

# For data visualisation
import matplotlib.pyplot as plt

# For random initialization of the trading strategy vector
import random

# For making waiting fun :)
from tqdm import tqdm

In [ ]:
def relu(x):
return 1/(1+np.exp(-x))

In [ ]:
#list of feautures that we have defined inside for the dataset
niceness=[]
retailiating=[]
forgiving=[]
non_envious=[]
cheating_sequence_length=[] 

for i in range(202):
    niceness.append(-1)
    retailiating.append(-1)
    forgiving.append(-1)
    non_envious.append(-1)
    cheating_sequence_length.append(-1)

def niceness_value(player_moves,opponent_moves):
    number_of_rounds=len(player_moves)
    number_of_trusts=0
    for i in range(len(player_moves)):
        if player_moves[i]=="TRUST":
            number_of_trusts+=1
    y=number_of_trusts/number_of_rounds
    return relu(y-0.5)

def retailiating_value(player_moves,opponent_moves):
    number_of_rounds=len(player_moves)
    number_of_trusts=0
    for i in range(len(player_moves)):
        if player_moves[i]=="CHEAT" and opponent_moves[i-1]=="CHEAT":
            number_of_trusts+=1
    y =  number_of_trusts/number_of_rounds
    return relu(y-0.5)
    

def forgiving_value(player_moves,opponent_moves):
    number_of_rounds=len(player_moves)
    number_of_betrayals=0
    for i in range(1,len(player_moves)):
        if player_moves[i]=="TRUST" and opponent_moves[i-1]=="CHEAT":
            number_of_betrayals+=1
    y = number_of_betrayals/number_of_rounds
    return relu(y-0.5)   

def non_envious_value(player_moves,opponent_moves):
    number_of_rounds=len(player_moves)
    number_of_betrayals=0
    for i in range(1,len(player_moves)):
        if player_moves[i]=="TRUST" and opponent_moves[i-1]=="TRUST":
            number_of_betrayals+=1
    y =  number_of_betrayals/number_of_rounds
    return relu(y-0.5)     

def retaliation_frequency(player_moves,opponent_moves):
    #return 0
    number_of_rounds=len(player_moves)
    lengths_of_cheating_sequences=[]
    for i in range(len(player_moves)):
        if player_moves[i]=="CHEAT" :
            length=1
            for j in range(i+1,len(player_moves)):
                if player_moves[j]=="CHEAT":
                    length+=1
                else:
                    break
            lengths_of_cheating_sequences.append(length)
    if len(lengths_of_cheating_sequences)==0:
        return 0
    else:
        return relu(max(lengths_of_cheating_sequences)/number_of_rounds)    

In [ ]:
import random
#more list of feautures that we have defined inside for the dataset

grudge_holding1=[]
randomization_tendency1=[]
adaptive_strategy1=[]
pattern_recognition1=[]
long_term_planning1=[]
adversarial_detection1=[]
cooperation_initiation1=[]
risk_taking_behavior1=[]
for i in range(202):
    grudge_holding1.append(-1)
    randomization_tendency1.append(-1)
    adaptive_strategy1.append(-1)
    pattern_recognition1.append(-1)
    long_term_planning1.append(-1)
    adversarial_detection1.append(-1)
    cooperation_initiation1.append(-1)
    risk_taking_behavior1.append(-1)
def grudge_holding(player_moves, opponent_moves):
    consecutive_defects = 0
    max_consecutive_defects = 0
    for i in range(len(player_moves)):
        if player_moves[i] == "CHEAT" and opponent_moves[i] == "CHEAT":
            consecutive_defects += 1
            max_consecutive_defects = max(max_consecutive_defects, consecutive_defects)
        else:
            consecutive_defects = 0
    return max_consecutive_defects / len(player_moves)

def randomization_tendency(player_moves):
    random_count = sum(1 for move in player_moves if move == "RANDOM")
    return random_count / len(player_moves)

def adaptive_strategy(player_moves, opponent_moves):
    adaptive_count = 0
    for i in range(1, len(player_moves)):
        if player_moves[i] == opponent_moves[i - 1]:
            adaptive_count += 1
    return adaptive_count / len(player_moves)

def pattern_recognition(player_moves, pattern_length=3):
    patterns = set()
    for i in range(len(player_moves) - pattern_length + 1):
        pattern = tuple(player_moves[i:i + pattern_length])
        patterns.add(pattern)
    return len(patterns) / len(player_moves)

def long_term_planning(player_moves, opponent_moves):
    planning_count = 0
    for i in range(1, len(player_moves)):
        if player_moves[i - 1] == "TRUST" and opponent_moves[i - 1] == "TRUST" and player_moves[i] == "CHEAT":
            planning_count += 1
    return planning_count / len(player_moves)

def adversarial_detection(player_moves, opponent_moves):
    detection_count = 0
    for i in range(1, len(player_moves)):
        if player_moves[i] != opponent_moves[i] and player_moves[i - 1] == opponent_moves[i - 1]:
            detection_count += 1
    return detection_count / len(player_moves)

def cooperation_initiation(player_moves):
    cooperation_initiation_count = sum(1 for move in player_moves if move == "TRUST" and player_moves.index(move) == 0)
    return cooperation_initiation_count / len(player_moves)

def risk_taking_behavior(player_moves):
    risk_count = sum(1 for move in player_moves if move == "CHEAT")
    return risk_count / len(player_moves)

# Example: Add feature values to corresponding lists

In [ ]:
#general function to apply all the features to the player
def apply_features(playername,playermoves,opponentmoves):
    if (niceness[playername]==-1):
        niceness[playername]=niceness_value(playermoves,opponentmoves)
        retailiating[playername]=retailiating_value(playermoves,opponentmoves)
        forgiving[playername]=forgiving_value(playermoves,opponentmoves)
        non_envious[playername]=non_envious_value(playermoves,opponentmoves)
        cheating_sequence_length[playername]=retaliation_frequency(playermoves,opponentmoves)
        grudge_holding1[playername]=grudge_holding(playermoves,opponentmoves)
        randomization_tendency1[playername]=randomization_tendency(playermoves)
        adaptive_strategy1[playername]=adaptive_strategy(playermoves,opponentmoves)
        pattern_recognition1[playername]=pattern_recognition(playermoves)
        long_term_planning1[playername]=long_term_planning(playermoves,opponentmoves)
        adversarial_detection1[playername]=adversarial_detection(playermoves,opponentmoves)
        cooperation_initiation1[playername]=cooperation_initiation(playermoves)
        risk_taking_behavior1[playername]=risk_taking_behavior(playermoves)
        
    else:
        alpha = 0.3  # adjust as needed

        niceness[playername] = alpha * niceness_value(playermoves, opponentmoves) + (1 - alpha) * niceness[playername]
        retailiating[playername] = alpha * retailiating_value(playermoves, opponentmoves) + (1 - alpha) * retailiating[playername]
        forgiving[playername] = alpha * forgiving_value(playermoves, opponentmoves) + (1 - alpha) * forgiving[playername]
        non_envious[playername] = alpha * non_envious_value(playermoves, opponentmoves) + (1 - alpha) * non_envious[playername]
        cheating_sequence_length[playername] = alpha * retaliation_frequency(playermoves, opponentmoves) + (1 - alpha) * cheating_sequence_length[playername]
        grudge_holding1[playername] = alpha * grudge_holding(playermoves, opponentmoves) + (1 - alpha) * grudge_holding1[playername]
        randomization_tendency1[playername] = alpha * randomization_tendency(playermoves) + (1 - alpha) * randomization_tendency1[playername]
        adaptive_strategy1[playername] = alpha * adaptive_strategy(playermoves, opponentmoves) + (1 - alpha) * adaptive_strategy1[playername]
        pattern_recognition1[playername] = alpha * pattern_recognition(playermoves) + (1 - alpha) * pattern_recognition1[playername]
        long_term_planning1[playername] = alpha * long_term_planning(playermoves, opponentmoves) + (1 - alpha) * long_term_planning1[playername]
        adversarial_detection1[playername] = alpha * adversarial_detection(playermoves, opponentmoves) + (1 - alpha) * adversarial_detection1[playername]
        cooperation_initiation1[playername] = alpha * cooperation_initiation(playermoves) + (1 - alpha) * cooperation_initiation1[playername]
        risk_taking_behavior1[playername] = alpha * risk_taking_behavior(playermoves) + (1 - alpha) * risk_taking_behavior1[playername]

In [ ]:
#parsing the csv file and applying the features to the players

with open("input_game.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)     
    next(csvreader)
    csvreader = list(csvreader)
    lastgame=-1
    lastplayer1=-1
    lastplayer2=-1
    playerstrategy=[]
    opponentstrategy=[] 
    for i in range(len(csvreader)):
        a=csvreader[i]
        a[0]=int(a[0]) #game number
        a[1]=int(a[1]) #player number
        a[2]=int(a[2]) #player number
        a[5]=int(a[5]) #round number 
        #print(i)
        if a[0]!=lastgame:
            if lastgame!=-1:
                apply_features(lastplayer1,playerstrategy,opponentstrategy)
                apply_features(lastplayer2,opponentstrategy,playerstrategy)
            lastgame=a[0]
            playerstrategy=[a[3]]
            opponentstrategy=[a[4]]  
            lastplayer1=a[1]
            lastplayer2=a[2]            
        else:
            playerstrategy.append(a[3])
            opponentstrategy.append(a[4])
            lastgame=a[0]          
            lastplayer1=a[1]
            lastplayer2=a[2]
    csvfile.close()

In [ ]:
#reading the data and defining the profit function
import csv
# Read data from CSV
games = []

with open("input_game.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header
    for row in csvreader:
        games.append(row)


# Define functions for calculating profits and comparing strategies
def calculate_profit(action1, action2):
    if action1 == 'TRUST' and action2 == 'TRUST':
        return 2, 2
    elif action1 == 'CHEAT' and action2 == 'CHEAT':
        return 1, 1
    elif action1 == 'TRUST' and action2 == 'CHEAT':
        return -1, 3
    elif action1 == 'CHEAT' and action2 == 'TRUST':
        return 3, -1

def compare_strategies(player_actions, other_player_actions, margin=0.1):
    total_comparisons = len(player_actions)
        
    if total_comparisons == 0:
        return 0.0  # If there are no comparisons to make, return 0 similarity
        
    similar_strategy_count = 0
        
    for i in range(total_comparisons):
        p1_profit, _ = calculate_profit(player_actions[i], player_actions[i])
        for other_actions in other_player_actions:
            other_profit, _ = calculate_profit(player_actions[i], other_actions[i])
            if abs(p1_profit - other_profit) <= margin:
                similar_strategy_count += 1
                break
    similarity_ratio = similar_strategy_count / total_comparisons
    return similarity_ratio


In [ ]:
[[0.5523284584837184, 0.4362985452900899, 0.49676323177319615, 0.43243360951143334, 0.5350665827798042, 0.08615526105969855, 0.0, 0.4530780834731858, 0.09064230108274501, 0.01180298287362037, 0.11821640905860023, 0.04835949451194478, 0.2875758416754444], [0.5260772016210652, 0.47107891523203693, 0.40400668469684065, 0.4950256343494236, 0.5385367943246011, 0.15552528722579786, 0.0, 0.8610498589187883, 0.10222966445743076, 0.011529861597077492, 0.13398118615911161, 0.6024517881896491, 0.39421604165232005], [0.5788873880202571, 0.41489723626935804, 0.40020520061505493, 0.5511353541968096, 0.4340805744269901, 0.015592898889577612, 0.0, 0.8614273480363532, 0.07780054138639822, 0.020908114295078592, 0.1461479630932239, 0.8204169959374201, 0.17941627583669414], [0.38618214256908545, 0.4985095488884258, 0.3815008932015086, 0.3821669586942299, 0.6380727215688738, 0.18224650567538186, 0.0, 0.5059259122974912, 0.06884510803051865, 0.014637996527903655, 0.13333459326409625, 0.00017634346737470234, 0.963443182759779], [0.4582986947468987, 0.45606286694853915, 0.4286739839045376, 0.4069218166382687, 0.583943389672061, 0.056091352912709926, 0.0, 0.43940760793840283, 0.09926427492987698, 0.039080466779938816, 0.22743394789803806, 0.000484129552873554, 0.6696754219725805], [0.5563592833839117, 0.41821497277781433, 0.48184406312657246, 0.4463440209630657, 0.5082987448162815, 0.030501567796024365, 0.0, 0.4516740933211083, 0.1023321989021579, 0.10445590826497712, 0.20521593097926621, 0.7262119578397054, 0.27355961255249006], [0.5696450200977486, 0.41722734287750707, 0.4778273242620452, 0.46888004819682466, 0.5204725539393045, 0.06772553275586096, 0.0, 0.5299975735141157, 0.07149627760839543, 0.035234955589512645, 0.14070757020156352, 0.0, 0.21755946125390502], [0.5580824292920935, 0.4118847936477024, 0.4667536558750758, 0.4637542369018265, 0.5064343233378918, 0.0215596955309714, 0.0, 0.49338236032832716, 0.09942267718663893, 0.1194240844040798, 0.22925921542780103, 0.6042136859820096, 0.26660935019682114], [0.6103509885286507, 0.37925408250077086, 0.44516592067153293, 0.5378055748893501, 0.5048492346380153, 0.016254381059452695, 0.0, 0.6600416262214116, 0.0862468042039013, 0.04366205720086694, 0.1082462051698447, 0.9486497900411437, 0.05119238617671214], [0.49936139872817364, 0.4593759970646604, 0.4587813405769764, 0.4163167408238895, 0.5269052500699727, 0.05294296322318047, 0.0, 0.4847356020317327, 0.12123521272092458, 0.050883518961007984, 0.2042809483350921, 0.11346162857439836, 0.5025735883928804], [0.47510422025210697, 0.4792270818059553, 0.4498401668059293, 0.4020767967977308, 0.5562475304311297, 0.04553602289519547, 0.0, 0.506503999870203, 0.1061509811726914, 0.031240266480689432, 0.2201462585631955, 4.617159444515433e-05, 0.6003603836362071], [0.5438752399616323, 0.4494069234177372, 0.43380967799157966, 0.4842287430157194, 0.5075518490376346, 0.01992204521718011, 0.0, 0.7251605724856182, 0.10360073907936959, 0.019788116673293846, 0.2777867085546607, 0.4183272484173276, 0.3232384157657339], [0.609672299693083, 0.3852137687463964, 0.4480755369540679, 0.5336654384931105, 0.4885161752129764, 0.009826300882519153, 0.0, 0.6680224216723556, 0.09220910824694963, 0.018050089705780387, 0.1641595309248024, 0.9459797433802113, 0.05405446755396371], [0.43129968401619134, 0.47320934823715033, 0.38134932621367457, 0.4248014500107927, 0.633261901969159, 0.10217565138515722, 0.0, 0.5845871409241812, 0.07579843779568818, 0.02939860356125428, 0.24871911333208457, 0.2061874160459707, 0.7798683629499512], [0.39109806121919055, 0.45831170638439644, 0.3816385081212822, 0.38674246991707883, 0.6148356261420859, 0.04896726958490887, 0.0, 0.3640723644769609, 0.07167701457985023, 0.02743356142925158, 0.25834097613448076, 0.004257203663769071, 0.9427289217171468], [0.5396108518616924, 0.38763203803719554, 0.5226716561439754, 0.3901005870696459, 0.5205708297534215, 0.021689384448214298, 0.0, 0.0955629421977843, 0.10379724198197987, 0.1306198031555565, 0.15744414760769287, 0.6599389127870599, 0.33981864695176417], [0.5315019729297592, 0.4323157500798185, 0.43646454915402644, 0.4675424421090689, 0.5149236606032273, 0.02294502744316903, 0.0, 0.5934113420548295, 0.1111309790276324, 0.018780391963183256, 0.23180376997095845, 0.6107679373511603, 0.3721389679686174], [0.516536766361409, 0.3821761060088783, 0.4845006927071848, 0.4049547143561265, 0.5509177858747427, 0.013318927732465048, 0.0, 0.13366437394443753, 0.11754128849090897, 0.10119957522810769, 0.12797120314374383, 0.5670113214290267, 0.43300572806609955], [0.5395313052177146, 0.41735204223915623, 0.40967637278584246, 0.5069789839216257, 0.5520948451656182, 0.015607305673379214, 0.0, 0.6869020273049575, 0.07272461873525898, 0.03519600770900518, 0.2610693471154979, 0.006067293235665171, 0.33979354320870614], [0.5562626225892668, 0.4390963615895711, 0.4170391693401604, 0.5108618329362414, 0.4869287605423308, 0.03200746378045054, 0.0, 0.7975102143902832, 0.10927744862047681, 0.019083130811128574, 0.19499953195874897, 0.72492459174879, 0.27257889582079375], [0.47389890207241225, 0.4707221418302572, 0.4472397310568501, 0.40322824879715247, 0.5470787942603275, 0.11361917601292688, 0.0, 0.47371404477255635, 0.1259305228875148, 0.09405980671867521, 0.186905039772536, 0.00213817061921602, 0.6062721195044], [0.44837309983648554, 0.483958938852205, 0.379706738886715, 0.4418415367834039, 0.5908210874205848, 0.26502419505486613, 0.0, 0.697964429352876, 0.08533436270769086, 0.02248780304022911, 0.12227588548449483, 0.29075391841086834, 0.7087380176689554], [0.41518157926519317, 0.4650054970104678, 0.3786251370244853, 0.4112670444910118, 0.6296852264294518, 0.0688998539515658, 0.0, 0.49856393835915463, 0.10821518374334935, 0.01920142581573743, 0.23734759581561782, 0.1415331421733581, 0.8439689419247111], [0.4694358357459228, 0.41624567482592933, 0.4361970679413678, 0.4093890020195303, 0.52537223026071, 0.02090593923520867, 0.0, 0.286229164937413, 0.10605892181263088, 0.170435746900308, 0.23075258767004597, 0.04313927562629329, 0.6227103824981239], [0.38976485279212847, 0.5166573702473631, 0.38635497362008764, 0.38093702246101446, 0.6092414728470318, 0.1978374568252687, 0.0, 0.5706348970564646, 0.06362338472180139, 0.01644334001266208, 0.16972077739724808, 0.0, 0.9484372687047866], [0.492092675865212, 0.44696510821221913, 0.42939800192914124, 0.4365891925239288, 0.5219885372206027, 0.03379571075751215, 0.0, 0.52462710559944, 0.1259007695705167, 0.12758159183128476, 0.2740136854538391, 0.23028885834207163, 0.531657041760368], [0.4724484583388681, 0.505932696277569, 0.4147489127747758, 0.4349307858404139, 0.5811048311877193, 0.21123443251926655, 0.0, 0.7441193660385502, 0.08492632177859012, 0.009840005028540367, 0.12178930856774606, 5.495808228394428e-06, 0.6122734703530837], [0.4812589383885907, 0.42186063867977, 0.4034226792791352, 0.4548025995976456, 0.5601340697392857, 0.028858548669503024, 0.0, 0.49500377006916857, 0.12963799731073058, 0.09320808043738324, 0.24090309857797562, 0.0, 0.5759738046881303], [0.47338150203782126, 0.46067922839322883, 0.4490226004307816, 0.3998753020750522, 0.5561181243181437, 0.10574127218177759, 0.0, 0.43185494028307825, 0.10119732376740659, 0.12379935305766519, 0.17150670146281824, 0.16400710779470545, 0.6088175425739031], [0.5041186028621355, 0.4838932536182055, 0.3795831432916623, 0.4977914822975944, 0.5543587966782159, 0.18431309346110786, 0.0, 0.9244039376885168, 0.1294513533457743, 0.02861366668415689, 0.07679292376492966, 0.4776968147169416, 0.4829070646589779], [0.5598054992449103, 0.39989083236198775, 0.4420830757720801, 0.4901389295403158, 0.5055663757586408, 0.017484409265973583, 0.0, 0.5532761977562486, 0.07766361221314938, 0.15775125369499216, 0.2262119476663117, 0.7386823080428765, 0.2596032275391519], [0.500940610406912, 0.4146842501464652, 0.4810918540458765, 0.3963702622305766, 0.5159351687687843, 0.029324302922523547, 0.0, 0.2223008717463042, 0.11281012279812186, 0.11826059341271696, 0.25552647165022035, 0.017806523056807542, 0.4960077471846793], [0.39697937452965487, 0.5012367713914814, 0.3885044971405654, 0.3857301454027303, 0.5903164341292705, 0.11819373470813785, 0.0, 0.530976972136652, 0.075959825436426, 0.03802098509167209, 0.25352129816317887, 0.0026080298039834545, 0.9182237646804289], [0.5422007965732258, 0.41090776689056613, 0.4588044705943085, 0.4548703372043198, 0.435081525066276, 0.071817981929867, 0.0, 0.45441656364966776, 0.10265119643832382, 0.01705537599233291, 0.10717892904785022, 0.6712897127509657, 0.3286993145212826], [0.5226207109961951, 0.42189893186273114, 0.42622815223165467, 0.46751666692838945, 0.540210362400802, 0.02260452750814624, 0.0, 0.5522460357893401, 0.1529646940630149, 0.02923230042802839, 0.20463214906774296, 0.5912449577955333, 0.4087884759913561], [0.46117761020949577, 0.44683372095347285, 0.3863402354963774, 0.4486282801508815, 0.5596939723895351, 0.0344512218084411, 0.0, 0.5752619026483444, 0.09839956516835714, 0.0313626725335533, 0.2332728725223111, 0.26641486729505287, 0.6568452992435713], [0.4431747859238325, 0.46529987916144855, 0.3812923754635539, 0.4354826881669708, 0.5793006627845698, 0.08300987693708382, 0.0, 0.5969095776198837, 0.08397035381069079, 0.02412677421008275, 0.14954337245190083, 0.26996826539450797, 0.7301587832115652], [0.4180144651904979, 0.44668280018134954, 0.38341138599569813, 0.4074598223174346, 0.6164647897826249, 0.07078336044553533, 0.0, 0.40713649682086245, 0.09822067939421746, 0.033400772421726246, 0.1392456458420001, 0.16705541374516306, 0.8329300908838917], [0.556980830108149, 0.4233983036421296, 0.43849378636704806, 0.49562412406116263, 0.5359614059360358, 0.05041330803319092, 0.0, 0.6644051805090556, 0.10607739951549928, 0.018737414399705853, 0.14203832494809226, 0.002644881313197522, 0.26884527319454876], [0.4915476374944463, 0.4403517172690943, 0.4346491737563173, 0.4296728300090786, 0.5220826320961257, 0.04239669159716265, 0.0, 0.47412627784461164, 0.13634310035955202, 0.13135713983676536, 0.2664507876402705, 0.41078878949549985, 0.5338399952538415], [0.4986856403962152, 0.4348032065805223, 0.4335027817214204, 0.4405549113231957, 0.5278586547116635, 0.04882058315881758, 0.0, 0.4898420375425231, 0.11203228416690027, 0.0692257468819275, 0.20205344734269615, 0.09631822397125404, 0.5056726373310929], [0.5161767849915273, 0.47480939244071874, 0.38082028251752853, 0.5087873195442009, 0.5267432410196562, 0.08621299434483533, 0.0, 0.9321407323080999, 0.12423425370977662, 0.023740850796302834, 0.1821796244215727, 0.5474392117624679, 0.43404518630129985], [0.5603941959759366, 0.3876218364076627, 0.4352517957008325, 0.49777504884243806, 0.5049244442146693, 0.010219816395193478, 0.0, 0.5325631168072289, 0.07509636863128938, 0.20826161279983615, 0.2362919214413347, 0.6894247920155286, 0.25723957011757814], [0.4758468347836043, 0.4403126744726122, 0.4061097455692552, 0.4462171884601756, 0.5511636349480581, 0.03839238179162194, 0.0, 0.5353775489997812, 0.11795707469466132, 0.10105435761191431, 0.16609754070763996, 0.0019047873401012302, 0.5968219418692583], [0.5375791368859181, 0.45876018597308316, 0.44893403715407376, 0.4606488942012864, 0.507047657039339, 0.026521034018344124, 0.0, 0.6753467125534842, 0.10144443810473627, 0.013539212773305527, 0.3143180102237222, 0.6510282881506697, 0.3489716645167154], [0.5235340684765991, 0.4477866869418256, 0.4523891837015592, 0.4431415769795195, 0.524546693289908, 0.02922989320277746, 0.0, 0.5573443246725283, 0.13429375694419504, 0.023362620574523804, 0.2898175553709488, 0.5939288885986909, 0.40524436441441203], [0.5255108732700897, 0.4261386975138186, 0.42815338068437725, 0.47008267995328845, 0.5299401452051439, 0.022336105011194988, 0.0, 0.5791487600417107, 0.11898875255374916, 0.009120995484454227, 0.19535827930205382, 0.6038340492669388, 0.39612055994948064], [0.4947777068651149, 0.4320602420674261, 0.42060441923928293, 0.4491336913625106, 0.5212262428951466, 0.05435479540850528, 0.0, 0.5143285059141801, 0.13556720601541639, 0.14498600255195912, 0.22603575120059485, 0.18395430933374646, 0.5209209479749881], [0.5069511405551593, 0.45040607452154235, 0.44677368785454397, 0.4353244360628581, 0.5154102381831605, 0.04056273895927713, 0.0, 0.5363863748681046, 0.13339300618732577, 0.09768214320887302, 0.22403253758816996, 0.099018388971982, 0.4721614955241895], [0.46410826173187925, 0.415508789020288, 0.4301973066836069, 0.4105284041718642, 0.5269682768647774, 0.024102015661764183, 0.0, 0.2852834737287381, 0.11180743425330542, 0.184159009183345, 0.2145246643661658, 0.0, 0.6441965130065135], [0.47055907160965316, 0.4314251713757671, 0.42588559564378703, 0.41736465395158434, 0.5707154030585202, 0.02062223447864874, 0.0, 0.38845027087922634, 0.12109990652543362, 0.0074077599338223995, 0.2099029015009675, 0.38061847064347454, 0.6185289542279395], [0.5203414946705442, 0.4452839528833487, 0.4488314981711605, 0.44430308123469736, 0.5316203251396406, 0.02660164229420764, 0.0, 0.5509253628856783, 0.11886942146512045, 0.00612277379819334, 0.2699566256729154, 0.5510318890186893, 0.4171916246355979], [0.5167714959175805, 0.42170679899405017, 0.39401504729154385, 0.49512354886956667, 0.5593821777690077, 0.05008368316219346, 0.0, 0.6622933270969811, 0.12787869574326316, 0.03961377086656555, 0.1438901808849481, 0.5683540930634291, 0.43165641964034146], [0.48560503291304224, 0.43183688236996864, 0.42485789601551377, 0.43530364962091583, 0.5279773383693425, 0.04988416001880067, 0.0, 0.4570832353106188, 0.14691976746913465, 0.15167069503423175, 0.2382457214206165, 0.18693650681078045, 0.557646526150739], [0.4716306314611165, 0.45631436283567106, 0.3894176738698742, 0.455819501878435, 0.4459923638429378, 0.0530711735622447, 0.0, 0.6420692386121498, 0.07210501313282046, 0.0070804027227462174, 0.21340538016264, 0.37532810387527804, 0.6140544588563239], [0.5163220225203875, 0.41331967278941517, 0.41453416538142174, 0.4741854862650976, 0.5720392576854113, 0.08440379286008518, 0.0, 0.5444491208720948, 0.09260783739400809, 0.024374272209840113, 0.12454734123200313, 0.56358781219854, 0.4344420551776158], [0.39498693728700074, 0.5473731758235647, 0.38696717820307625, 0.38297616470304185, 0.6195319202155745, 0.22798297877986629, 0.0, 0.7140939577857028, 0.0711879030143973, 0.010637725378393472, 0.1435018864740625, 0.061610946292202616, 0.9268123015650584], [0.5585755818469942, 0.39905058970375884, 0.4672770496953794, 0.46445676023150917, 0.5456901766651319, 0.03304400962508549, 0.0, 0.4443351125972939, 0.0965973868001764, 0.014402857214290082, 0.09545367275007417, 0.7182737521911089, 0.2631518787156834], [0.4937719657806683, 0.46261351373260295, 0.4709553410209796, 0.39953325444635435, 0.5604588848990797, 0.2129297676762515, 0.0, 0.4296390588819736, 0.11229954390389332, 0.07381623697519282, 0.14415147580072682, 0.017794095115478182, 0.52551507741703], [0.6140783386603444, 0.3798783482391287, 0.4485153552893827, 0.5384223196837976, 0.505129714491907, 0.007816479303648574, 0.0, 0.6657899085687969, 0.06471450702547325, 0.023643009146567452, 0.05777819461990967, 0.9635915182239506, 0.03546179304004776], [0.5604954939889377, 0.40329724571414977, 0.44910806654098134, 0.4831635511007445, 0.5061777691342877, 0.01622642438369084, 0.0, 0.5393491495175899, 0.0933292384925828, 0.14078545036166, 0.23217783499877453, 0.7078849860276557, 0.2568128326052271], [0.4583361476719595, 0.4351355691406422, 0.41867845230969, 0.4162162245013101, 0.5430129425570115, 0.06990810220541946, 0.0, 0.39154501815272313, 0.10738384287220505, 0.12377416210704639, 0.18929444201275505, 0.0, 0.6686828819739276], [0.506674616222776, 0.4452326568489416, 0.44129914327157893, 0.43743626096847454, 0.528384663505829, 0.022536703112180385, 0.0, 0.5261464166797618, 0.12491390210336922, 0.015376373045178884, 0.28272625746774155, 0.5271321635222433, 0.47288640022910877], [0.4357006504741952, 0.49935403127994127, 0.38174978211744554, 0.4275946103607501, 0.6117215462218085, 0.2031981923983036, 0.0, 0.7007411540622506, 0.09237296933674718, 0.008952217247877835, 0.12387029872382482, 0.23542103542564952, 0.7610810323410515], [0.4372333416794696, 0.47920776753143474, 0.38677541915266533, 0.4241306036257875, 0.6102236026945237, 0.1004578795255527, 0.0, 0.6072905724345383, 0.10940139870411356, 0.019718792974183663, 0.21377562316695925, 0.23737337121396868, 0.7540583387461416], [0.4928531801225698, 0.45306602019317044, 0.43873326250052697, 0.42918063397993467, 0.5228154752932602, 0.04568794018610515, 0.0, 0.5163815227060594, 0.14251719812395683, 0.10077122033224963, 0.24167283690344882, 0.14109307346722721, 0.5285969810919784], [0.49957635749553686, 0.49413132363750545, 0.38239240827818693, 0.49035612864008243, 0.5347057665925622, 0.08118841078968009, 0.0, 0.9372436704634127, 0.11486787317300717, 0.016732105144393106, 0.2761423925603774, 0.4984273370788018, 0.501600570593326], [0.48578509672119896, 0.42072654455960534, 0.4584093863447095, 0.40391775812139885, 0.5165694561973465, 0.03387419270022432, 0.0, 0.2877000504205754, 0.11834959525310795, 0.1827311564186348, 0.2518784366098017, 0.004966455705676964, 0.5569125474236208], [0.5507417410319544, 0.4437673225549852, 0.44224789214724647, 0.4799925719130319, 0.4314007023648021, 0.017926422617495205, 0.0, 0.692872616437828, 0.08602080993636357, 0.021336182570655823, 0.259853285660162, 0.6900826346864475, 0.29508800558787573], [0.5121955890757486, 0.42552078415577077, 0.4269498622773098, 0.45758338551474886, 0.5283382208945527, 0.016593536992054677, 0.0, 0.5273177014838918, 0.1127296866038128, 0.023250819030849367, 0.2137751016520754, 0.5492423689335215, 0.45078375994049874], [0.5107511188392767, 0.42206636188441227, 0.4992992569772038, 0.3885429898422103, 0.5315958948084027, 0.035147687666950755, 0.0, 0.21692861186653872, 0.15195388030322893, 0.084272256542059, 0.211196348354968, 0.0005177539512786832, 0.4570618448943692], [0.49810542552268133, 0.38067761562576774, 0.48536811250101275, 0.38621998693368065, 0.5420730607273174, 0.016999060223630583, 0.0, 0.0497850375625016, 0.10961187803721911, 0.1838509529584898, 0.18352968691096275, 0.49194122481368097, 0.5080872090740283], [0.5703202078845314, 0.40925356132579427, 0.4540281965927021, 0.48877420486546563, 0.39343480155431226, 0.09361931592289824, 0.0, 0.5856620312185633, 0.07771359949864866, 0.007053811255771779, 0.06820841949801225, 0.7846174338993942, 0.21464378306730608], [0.5414568589007367, 0.45296828683442747, 0.43528637428905526, 0.47820468059149723, 0.507762320175484, 0.02560438208725618, 0.0, 0.7220842762377828, 0.10625663942612888, 0.02073910907517933, 0.2508881994798087, 0.6582260657542309, 0.3329185354790096], [0.495449310588016, 0.44628701583053143, 0.4368606662756669, 0.43489169326310906, 0.5280435018577697, 0.03801845019149763, 0.0, 0.5103023178100377, 0.09900744169147166, 0.05730251340302634, 0.2019138739574545, 0.0, 0.5184856911550068], [0.4828382701597064, 0.4779279076775983, 0.41580522847173274, 0.4444347794212392, 0.5717316410327413, 0.08267241582543648, 0.0, 0.67530188415874, 0.07864794554534435, 0.018254988138783276, 0.1779930515327845, 0.00029021477481702506, 0.5690663270461511], [0.4995246757265153, 0.4926962951337741, 0.38122692120429447, 0.4910589432139847, 0.5494289857039336, 0.18510274530678011, 0.0, 0.9342346415806274, 0.12824803092440232, 0.025399053868280074, 0.1892473892442602, 0.4976771521311169, 0.5022924589583653], [0.521165750958849, 0.41706820473092754, 0.5109349607611858, 0.3846626200599622, 0.5518169472975625, 0.15646597843477425, 0.0, 0.18882789845986916, 0.0817205961090773, 0.10975774795482265, 0.09796848978698933, 0.582148575359658, 0.4146025268657655], [0.5146180860003772, 0.47691460453204815, 0.38184208849271484, 0.5063559942461935, 0.5361135001365445, 0.13519767252812132, 0.0, 0.9326791379735371, 0.11712006847945798, 0.023729876170658352, 0.14198800434998984, 0.5116066331729702, 0.4412696070367953], [0.5300766277843496, 0.4651043322908215, 0.4522979530194058, 0.4501947197368019, 0.5069166881155528, 0.026429140874050357, 0.0, 0.6560580591844762, 0.09097584398093284, 0.018875466121844056, 0.33180414913387446, 0.6212483373042982, 0.37872033295445773], [0.4638637962297585, 0.3787629215966569, 0.4328057394631243, 0.4033375823437966, 0.5698016258398195, 0.016166313832831153, 0.0, 0.11286183912293422, 0.12486804625441283, 0.14888931616259615, 0.23136271886156767, 0.3524981214019841, 0.6458930283380555], [0.5073465795533185, 0.43338903368548065, 0.4301763626641424, 0.44999375864904906, 0.5429541508488711, 0.020895326599005414, 0.0, 0.5273811956909634, 0.10623720638703542, 0.010896484137141369, 0.22533094637015882, 0.5294377452819714, 0.47062524830193353], [0.4815279784311899, 0.42403757214234333, 0.4540154071877291, 0.4042783134336792, 0.5334177477110444, 0.020309820077609755, 0.0, 0.29550958433309293, 0.10428071266732808, 0.1471682415873822, 0.22160352597253866, 8.230840920177947e-06, 0.5741051804197931], [0.4343249821419072, 0.5025559267700068, 0.4113150473663486, 0.3999911479071121, 0.5984366101980463, 0.2823886602299804, 0.0, 0.5913612556814004, 0.11764546289610843, 0.0716177601507827, 0.14510566110976078, 0.005282688449548831, 0.7666973884931405], [0.3978321144715656, 0.5131842944636216, 0.3831338975308054, 0.38898849274704905, 0.6515752755920766, 0.14548264593703814, 0.0, 0.6014681181353696, 0.08174570543385246, 0.022354307613537554, 0.15505217253527684, 0.07704466940038222, 0.9153224622353713], [0.5789316518252475, 0.4095441000568746, 0.3985889006580827, 0.5533286821876413, 0.5066210254986117, 0.01137006216508937, 0.0, 0.8476406645542275, 0.10289417589968466, 0.03799360195919888, 0.12208688592928714, 0.8168109047631581, 0.1803828682811791], [0.4985596846790744, 0.43536314112572394, 0.4536633716877554, 0.4211934842870984, 0.5742227165804291, 0.01985493875019546, 0.0, 0.4151848067617009, 0.0752365510996845, 0.008450499075267352, 0.24623240335135588, 0.2804440709645994, 0.5059376502612343], [0.4832737373359929, 0.4275109433044876, 0.4030572840900589, 0.45264618249019944, 0.609334481236001, 0.043612700602264756, 0.0, 0.5132317048242374, 0.10695196946386074, 0.026979894412867685, 0.2016186112880789, 0.42944093485390245, 0.5675562400699118], [0.5671553502812542, 0.40784189962748013, 0.45395799623287636, 0.4866072177313884, 0.4748897911500264, 0.06082709821867484, 0.0, 0.5724910071164871, 0.08567925833321578, 0.008499215143638602, 0.13097841801885624, 0.6391407578158002, 0.22854474449368162], [0.4878966595618551, 0.4351575283939455, 0.43143765919577604, 0.43187155355315754, 0.5273922978971815, 0.04562274602485649, 0.0, 0.45277057960006284, 0.146495707987979, 0.15227653196923377, 0.24940423429531172, 0.11869329193657309, 0.5485764050633479], [0.48547259259858, 0.435350389312523, 0.4484086561849537, 0.41334524026647224, 0.5451702226389933, 0.08203991276362133, 0.0, 0.3788003116882099, 0.12644613539838678, 0.10558496709072769, 0.20157633758721294, 0.002274782659112397, 0.5584865075884183], [0.4755501689641859, 0.4270995444382467, 0.42699179623489325, 0.42132548522386903, 0.6261954066857497, 0.060960474363225325, 0.0, 0.3839122027091085, 0.10980038217033196, 0.01128450624277211, 0.16158177578782829, 0.3908981563968032, 0.5991817348551551], [0.5449176122899622, 0.4474199130502742, 0.38199518671314503, 0.535656934756415, 0.5156978540669048, 0.03895251698143845, 0.0, 0.9309360157593095, 0.10782941107730237, 0.02611221739838143, 0.1487156130667543, 0.6817816641390119, 0.3182318512777951], [0.5000187662279864, 0.3865172535977959, 0.45506874883635573, 0.4184437035480637, 0.5304108919870723, 0.02022249508205351, 0.0, 0.20103118393776537, 0.12669577552808253, 0.1276125987443985, 0.22061884705635637, 0.3928146141326627, 0.49964421259392133], [0.6140740920257597, 0.38121271090321074, 0.4670651256596451, 0.5214966161174445, 0.45181172757456156, 0.007517709049088983, 0.0, 0.6030924289244615, 0.05369241797405824, 0.0170596987221889, 0.08354981272977932, 0.765857405082413, 0.03552090255499646], [0.40170415641998214, 0.49615033759507204, 0.38358056826785714, 0.3922800147036787, 0.6257193547729576, 0.15808781147540737, 0.0, 0.5461339569622549, 0.0743207394071164, 0.01284953117653534, 0.15684634703911973, 0.09631713529758941, 0.8993397183749434], [0.4019837650145124, 0.5530070133112488, 0.3843153258974862, 0.39159730549246385, 0.636858104987073, 0.23198895490764704, 0.0, 0.7732399020388854, 0.07980054024426099, 0.008518658287710422, 0.12659249383103863, 0.10243365657691582, 0.8975932237335842], [0.5010893950591061, 0.46997656164753654, 0.4682931651146668, 0.4083641922594694, 0.5199714368181184, 0.05691663342167218, 0.0, 0.4954043651782961, 0.13821881928187144, 0.05502647889699902, 0.2585881428107158, 0.18598161271405653, 0.49562963755341627], [0.5604848242475284, 0.3994003140375285, 0.4423404279679677, 0.4897241345926203, 0.5066840229386802, 0.02420514732888524, 0.0, 0.5497690893822027, 0.09208358507505375, 0.15723582932107966, 0.23435403746572725, 0.7431391559716245, 0.2568715709733035], [0.556613106593617, 0.4387748349672176, 0.4037758102095876, 0.5251027959394207, 0.3633108838621975, 0.06151912909611016, 0.0, 0.8531559548778209, 0.08288140266716763, 0.016018641166174193, 0.17569613142246904, 0.7159212360467537, 0.2694003798936692], [0.5766856877555264, 0.41268219614751533, 0.4019824636131301, 0.5474296049332987, 0.5064838203666064, 0.012907561412040067, 0.0, 0.8376502951059437, 0.10055329858439724, 0.04012937874696687, 0.16355889002856808, 0.8074926267216327, 0.1902597415018243], [0.5019580874513702, 0.4477901983055708, 0.4464682450815982, 0.4304060596403776, 0.5234158020213263, 0.0628612136677498, 0.0, 0.49720375822508894, 0.1473933884754384, 0.11021773267550891, 0.24892210683289184, 0.22612336575697772, 0.49218242570168447], [0.5597138857025901, 0.43279826985031367, 0.41499604359665454, 0.5173334589076497, 0.5064269184235316, 0.01919845952859166, 0.0, 0.798484906809759, 0.09667810622483494, 0.025356553925921545, 0.22590276003693024, 0.7405365704090552, 0.259521518002767], [0.6076895503190526, 0.38369947068885124, 0.5120788258221802, 0.46851612365338247, 0.4787239616649406, 0.014165557509429372, 0.0, 0.3987117702623295, 0.06377491654481644, 0.03623208532912775, 0.11079421859913002, 0.9376692021318217, 0.06233884427989614], [0.41555573277903, 0.48931877288850223, 0.3810091828181369, 0.40866033870567786, 0.6206865024105406, 0.07759900049084695, 0.0, 0.5864556155832408, 0.0972773153953237, 0.021318145315241496, 0.267863343648401, 0.14933962504846013, 0.8420034730574669], [0.5058213696779743, 0.46027576650785984, 0.46284640576221947, 0.4193989439576331, 0.5229207821717826, 0.05420378352795888, 0.0, 0.5005938005959499, 0.11643979826768917, 0.04701170949147378, 0.18202370391188996, 0.0, 0.47670286708051657], [0.542495037257486, 0.41554228853376396, 0.4166850919773108, 0.49887965409040635, 0.5203740895013513, 0.01732107136218957, 0.0, 0.6522066656170815, 0.10077845424124117, 0.018222772943369197, 0.16830900200785745, 0.6726212710138824, 0.3274587359208132], [0.3891890415556869, 0.5232415101776229, 0.38676218255576267, 0.3798893726884801, 0.6305196042707846, 0.23863636984829914, 0.0, 0.5904756562189786, 0.07696848165018955, 0.009935085026016192, 0.09366380476651746, 0.0009519141135554927, 0.9507998881472586], [0.46804526240120903, 0.4975381119184469, 0.4416012982376146, 0.40307752413391007, 0.5713035906152215, 0.2517874032733126, 0.0, 0.5880999239592672, 0.10149614410018794, 0.06878275017138515, 0.1317101545954916, 0.0236222882140953, 0.6296225487945651], [0.387056994039808, 0.4810327886088599, 0.3824353606614066, 0.3821479040840059, 0.651548421798178, 0.12372929942314657, 0.0, 0.43457969259288637, 0.07102220052008458, 0.01609980504526337, 0.2295451987757609, 1.3719598437812656e-06, 0.9597953137303629], [0.3904269958826964, 0.5155008305286015, 0.38346215925293453, 0.3844216818720766, 0.5988494875618355, 0.16846328398789323, 0.0, 0.5824587737532014, 0.06763703741607592, 0.027573817140143758, 0.11149364073576455, 0.0022217383479313443, 0.945594263333453], [0.4920792108465316, 0.4366696346081571, 0.4290121939909924, 0.43925424047533845, 0.5355809049672352, 0.04439160626364014, 0.0, 0.4882273887815702, 0.11883341266329803, 0.07813461784179918, 0.20751169170854888, 0.07162631727045898, 0.5320025945130162], [0.5005393165966708, 0.4935379909950087, 0.3800597079646923, 0.49420822939820186, 0.538639737747178, 0.16616504073465121, 0.0, 0.9494212539047331, 0.09913074655402238, 0.018403968685627554, 0.1392699707408289, 0.4530612761341284, 0.4980527912219664], [0.49358747013398296, 0.4246143903464513, 0.47423968766040414, 0.39647393035440937, 0.5181192565001811, 0.034992311066902426, 0.0, 0.25797111647452864, 0.13042582280126605, 0.13578401677364096, 0.2303179637540535, 0.0, 0.5259857451564722], [0.3915390755960231, 0.49057518964944247, 0.3851308101302939, 0.3839173201420007, 0.6019569096943568, 0.1743693626355905, 0.0, 0.48169431034707744, 0.058935890622320075, 0.0298918102363789, 0.18712224293941285, 1.9643823251426216e-05, 0.9408917295989832], [0.48913356650626516, 0.3814636758483214, 0.45369839593100253, 0.40785786977836463, 0.5420096980318684, 0.022509575683304542, 0.0, 0.14251430228420495, 0.1192185931221704, 0.15494764131000446, 0.20660653547547186, 0.4493489005622852, 0.5441715453381202], [0.484549032542379, 0.4512917013947748, 0.45495620893062305, 0.4060996537687438, 0.5486036668074785, 0.10982710206706105, 0.0, 0.4046621598495222, 0.13424640853622594, 0.10409504606494, 0.18624960016473818, 0.0, 0.5629745401652207], [0.5826686725329543, 0.4064738904938773, 0.3941986840358611, 0.5611215576460303, 0.5235252911338582, 0.07793281440489315, 0.0, 0.8650680154896943, 0.09700959332270442, 0.04523617037282031, 0.10726985307394546, 0.8332634472718127, 0.1638706726065833], [0.4845519294695403, 0.4237246432362568, 0.4548974964491347, 0.40592219699776666, 0.5187432892827002, 0.034242881055660555, 0.0, 0.3014145970955382, 0.11748746553782123, 0.15523084288288969, 0.23189425070550743, 0.01059314326146104, 0.5621547751039195], [0.4858764978761342, 0.4480825385605283, 0.4237920458943961, 0.435948097593055, 0.5256955099400371, 0.04328015284665174, 0.0, 0.5314965720365898, 0.14141900979250205, 0.11239658576714938, 0.25069278344188983, 0.23824645237561043, 0.5565777961234426], [0.6073007988064726, 0.38934822206073016, 0.4998034423179455, 0.47977654580826523, 0.5053086131070831, 0.015268999952156763, 0.0, 0.46837484946198754, 0.08964138278049263, 0.01367966566139317, 0.13053190645260127, 0.9359895077959837, 0.0640060655982531], [0.5183213154995163, 0.3872874931002607, 0.5080035276289989, 0.38483361514653336, 0.5219122036672751, 0.02407910321178137, 0.0, 0.06556461798445817, 0.11118830449440464, 0.0981415502152156, 0.13595668588950757, 0.3177427146523258, 0.4257548105825126], [0.4952895661406354, 0.4485119884873975, 0.4484771786007733, 0.42218020593075356, 0.5204969287423126, 0.037974077466503685, 0.0, 0.4665536050377027, 0.15236842565179062, 0.10805052208456058, 0.2780774047773308, 0.14044816929616769, 0.5189041809605461], [0.5001674877040978, 0.4190479216966889, 0.48202793342641775, 0.3951314603595115, 0.5285645680490251, 0.03016033030763362, 0.0, 0.23091957999108856, 0.13276924069625917, 0.10745219761390636, 0.2295255217431412, 0.0, 0.49918438507963514], [0.5642008635779638, 0.39900103854976054, 0.4361578163090891, 0.5051822523628555, 0.5306148862542213, 0.015494087487334786, 0.0, 0.6038038942832074, 0.09302683550991647, 0.03131180960698503, 0.14558393910347878, 0.0, 0.24010055240094721], [0.5662887304562245, 0.4208230804480686, 0.4042168006870551, 0.5352635082255733, 0.4941322210396586, 0.06716839185889471, 0.0, 0.820127319076781, 0.07831831610285014, 0.04975734621002258, 0.13107623664293583, 0.7693874736329941, 0.2306137561071854], [0.5575992343558251, 0.41139114372539576, 0.47617753912739236, 0.4535917051148935, 0.5065810485734558, 0.020244083311157412, 0.0, 0.4533918618170127, 0.09822577109282786, 0.12135766981625012, 0.21089696612060121, 0.7314308849666499, 0.26857170617197956], [0.6098168174886778, 0.38208830722097176, 0.48672385514486427, 0.49659794781921734, 0.5052572219516698, 0.018696603687488128, 0.0, 0.5054380948604942, 0.07822547765285147, 0.03229631996450675, 0.09829408574721783, 0.8475045417596222, 0.053428345615186196], [0.39383038210596255, 0.48474970186474897, 0.38697619291531127, 0.38434503148802573, 0.6145502038156428, 0.15954973574537204, 0.0, 0.45896368906396556, 0.0767938723497797, 0.03611655886547787, 0.12791552529089195, 0.0, 0.9313130166701094], [0.495060461605932, 0.4703977162324895, 0.46276365342190023, 0.4086894752842862, 0.5266782666351434, 0.056187173855819846, 0.0, 0.4944776597440349, 0.1258719443266617, 0.03091661509980591, 0.19351205608221278, 0.06853231437339448, 0.5198881179816465], [0.4753729161535085, 0.44954708729539283, 0.3910778322429181, 0.4573023116657118, 0.6109848528666261, 0.04024898230283795, 0.0, 0.6227981109962542, 0.09276616593895665, 0.027684518093499064, 0.25585104370593903, 0.40101606157794073, 0.5989857559625763], [0.46098463327342043, 0.479340075530218, 0.43902636878468365, 0.39868986281851065, 0.5703067753418175, 0.2100724900494246, 0.0, 0.49711543703182104, 0.10242372436029323, 0.1101300483018027, 0.16284594254255236, 0.011044759799039659, 0.6590502033258046], [0.5594999370983597, 0.4247151100268536, 0.5141689127601088, 0.41832811532644354, 0.5063543517479293, 0.02240288329894725, 0.0, 0.3643992183976026, 0.08714279475448185, 0.06285984945043972, 0.2385218878376183, 0.7365998405201964, 0.26087070647993027], [0.5033299457180431, 0.44171417266785007, 0.4448689763401466, 0.43190727201275814, 0.5177526915941872, 0.021234002052658366, 0.0, 0.48831024279684443, 0.11500707073224029, 0.01804722427079808, 0.2700274087477023, 0.3067766559678174, 0.4869855822458508], [0.49087003889762515, 0.46740338963577244, 0.4566179290486966, 0.40969389114775134, 0.5284132932321929, 0.0491173810190214, 0.0, 0.4931022148989084, 0.10053044114481258, 0.03362541220900286, 0.22264359270747713, 0.14345300260660004, 0.5367586858466328], [0.3865445998140192, 0.49355502567055887, 0.3821953501440044, 0.38183852204424634, 0.6336327961472218, 0.1127660236716083, 0.0, 0.48534336971907904, 0.05817612663011326, 0.02302497015892115, 0.12119310957435452, 0.0004326287792682035, 0.9618979792503091], [0.6116753963100379, 0.37952493446806723, 0.485219443019568, 0.49975563741537843, 0.5052249973150729, 0.009091882739381923, 0.0, 0.507649008890337, 0.07484975533353692, 0.03262556768956198, 0.13782527279305115, 0.9517993359021304, 0.04560725506053009], [0.503781021188996, 0.4912546113965808, 0.4019761255562981, 0.4745412558970046, 0.5343012458610525, 0.2048107850059509, 0.0, 0.8612778761969746, 0.09918985787833032, 0.014672383079634543, 0.16409473893260745, 0.5161037134440727, 0.48400749712438706], [0.5579986499228937, 0.4051962670795256, 0.4631227413543295, 0.467825740741389, 0.5059935590628403, 0.016461361952662186, 0.0, 0.48402199437834525, 0.07743656482873115, 0.13965391099677246, 0.21806121866640885, 0.6796589665492789, 0.266939155351897], [0.4380017745473189, 0.4922462741546651, 0.40807529962149025, 0.4067870537816184, 0.6015139318105738, 0.35444139288724696, 0.0, 0.5824859239813351, 0.10881754703389512, 0.08951304316985839, 0.12601750111125337, 0.01353481163761766, 0.7510483232107003], [0.43234406124659364, 0.48471962587525375, 0.38685182667779244, 0.4202924174751313, 0.6221057412255365, 0.07706531774621789, 0.0, 0.6139333036303916, 0.10148399899198654, 0.028818244625515847, 0.18556888267663638, 0.19587307802550935, 0.7741579120173714], [0.5822479548739392, 0.3809005267458595, 0.5335318585783956, 0.42123332001315783, 0.5119705541246954, 0.021625000939212415, 0.0, 0.19258288368569504, 0.11467148857542513, 0.08207227546715702, 0.1340409783000473, 0.8327663860512707, 0.16701605216132587], [0.5409436589570619, 0.4352085174086005, 0.4230901830647865, 0.4946063443885599, 0.5350332948424606, 0.06979049872288806, 0.0, 0.7094059303781107, 0.08063694327606664, 0.02888763445530464, 0.2112497684947683, 0.06701803383585414, 0.33373727061492], [0.5640455602293948, 0.4047445633156696, 0.46008785357384163, 0.4764858321867687, 0.5496481890124114, 0.0715856575603989, 0.0, 0.5179662732806423, 0.09534352378407401, 0.01371663942981348, 0.09279596531774643, 0.7577014911983518, 0.24168897752241214], [0.5067847910160829, 0.3831076243780023, 0.48237763197088435, 0.39754579119917055, 0.523080027346835, 0.01587038833616319, 0.0, 0.10619879776906657, 0.10890048143787698, 0.15019569717099726, 0.244207198953341, 0.5271054228463126, 0.47306164054890903], [0.4832554804521125, 0.4292264746856087, 0.3799115718350774, 0.4778015797545737, 0.5251860902056138, 0.08871353897643851, 0.0, 0.6220381753151133, 0.10720110242037345, 0.020182363475155448, 0.09154677010873707, 0.37676365018917557, 0.5677976657944086], [0.49223276528385285, 0.44607693025995215, 0.4171723641772902, 0.4490260401660207, 0.5219411790499245, 0.02634247271654587, 0.0, 0.5729547059822708, 0.1274403666213986, 0.1296072203136386, 0.2911765466490626, 0.24601526307890076, 0.5310881703727335], [0.41958758504462446, 0.4916005511091651, 0.3839159419386995, 0.40976158519286043, 0.6229646512455193, 0.11817221578722908, 0.0, 0.5981587912115719, 0.08943126941575122, 0.025310361909481015, 0.19297679758703085, 0.13222312272174494, 0.8254961944609466], [0.5695411376868843, 0.4237494594242337, 0.4009870091158846, 0.5405772760754177, 0.4090908150216888, 0.06161297901106709, 0.0, 0.8552231301027847, 0.10642551516752242, 0.02723966796806919, 0.1195437628806115, 0.7776997087092496, 0.21757721603897417], [0.5187623672667703, 0.47694050237730173, 0.40754860593109304, 0.4834303283986452, 0.5318987341409158, 0.11094090781177249, 0.0, 0.8374537120342604, 0.12629568635625252, 0.017847514677749948, 0.1439592553020923, 0.5771883091684183, 0.42287304726083386], [0.5009332672428749, 0.45734599884769167, 0.46201950370641576, 0.4135528889704803, 0.517770280317899, 0.04758052872383182, 0.0, 0.4735002002924197, 0.12255219720885323, 0.08313864883844614, 0.2602757246159362, 0.19924535778410005, 0.49626914544375655], [0.5361897549187202, 0.40316391165838983, 0.4635925731624501, 0.44534677378688814, 0.5419441225110422, 0.0509715109083068, 0.0, 0.3846903843044002, 0.09634457807377557, 0.019395289120855596, 0.1460655129700837, 0.6301110642427663, 0.3536511987599985], [0.4980614783802715, 0.42918950751878415, 0.4235263296364309, 0.450821746688248, 0.528399983560838, 0.03965188154517943, 0.0, 0.5097963784553159, 0.12406023029920814, 0.0932020893990574, 0.15741982958530873, 0.0, 0.5079442119230404], [0.5511121393459144, 0.4384889491622447, 0.4252025635768629, 0.4988115238314216, 0.5074229369260991, 0.021112556633562193, 0.0, 0.7448902566040553, 0.10335848871215375, 0.03751776175783491, 0.23658012785728152, 0.7062608886134428, 0.2937363767728133], [0.3923686911242754, 0.4672144313501434, 0.383672182493769, 0.3854052762993275, 0.6176645755400756, 0.07700814572591914, 0.0, 0.3956677749486096, 0.08652322509088652, 0.0353295924611404, 0.2663437706956422, 0.006535775430605007, 0.9375141199341503], [0.46578212447794276, 0.4038783418113179, 0.4495591415273479, 0.3930849584610937, 0.5429500596191029, 0.022230367836322224, 0.0, 0.16896524649637415, 0.13208676804557568, 0.1109324501269421, 0.19267935958025784, 0.0018337097560880951, 0.6383284706802111], [0.4499264373919834, 0.46529196395557715, 0.40589438954172286, 0.4174493104815288, 0.6276092041779098, 0.13561495298833168, 0.0, 0.5228578776428178, 0.08908572455167833, 0.01986112728683277, 0.13796837010693883, 0.29272381120019314, 0.7023783074349378], [0.3901778206211387, 0.4813669832015032, 0.38162882884103044, 0.3860683287271582, 0.6078873030170212, 0.06704892411994803, 0.0, 0.4492736025123485, 0.09125548181353911, 0.018154071014221, 0.29095376178928284, 1.9115990947058594e-05, 0.9465998182137847], [0.5124578770002675, 0.47915909898096143, 0.38516059361524463, 0.49988645062139203, 0.5167107091592962, 0.031235124430286254, 0.0, 0.915971654354993, 0.13673068348697762, 0.015207070798218201, 0.27689645038656285, 0.5500457763606609, 0.4499575197333525], [0.5109600122434625, 0.3803729117389024, 0.497417199664993, 0.38699080376710504, 0.5304948705384042, 0.017279318738911618, 0.0, 0.0517089298893819, 0.11255019673158323, 0.14187835109494348, 0.1865732059867019, 0.5376838871847269, 0.456191572952036], [0.5709997779591803, 0.4217382364777405, 0.40777278185623345, 0.5356611717816256, 0.5053456945752491, 0.01895840345203735, 0.0, 0.8267400717226513, 0.08736989339806281, 0.028342214621602867, 0.17333706730089798, 0.7866886053986288, 0.21324916314329417], [0.47424493850202276, 0.48030275059405003, 0.4505256455829076, 0.40040982161638283, 0.5845476908037485, 0.18708524882873256, 0.0, 0.5062087567520619, 0.08974554685351908, 0.006654157414492125, 0.11917289108901188, 0.04164675715727468, 0.6046291398231138], [0.4765784769414845, 0.45166943613213684, 0.4232408553410046, 0.4296461589038627, 0.5427984390389589, 0.034095960646403994, 0.0, 0.5063863234066242, 0.09368451242220366, 0.04639549668160267, 0.24969941797766554, 0.02121758771781135, 0.5949204003999949], [0.5034070121305814, 0.43707367584190626, 0.44098348541648646, 0.43811262491353253, 0.5261551457106673, 0.03396620645590395, 0.0, 0.4895043521921476, 0.09090902766653763, 0.03961993504700563, 0.18877108423035932, 0.0014693520391744488, 0.4863867438593042], [0.5176302954509818, 0.44912822508720124, 0.44918800570929757, 0.44069843163406675, 0.5157554766201016, 0.019975456366823213, 0.0, 0.5546163916493728, 0.11511527594953998, 0.01151742410970595, 0.28814838424092915, 0.570201114586118, 0.4291921800299355], [0.5030267537718723, 0.38097761807535924, 0.4535825260604236, 0.4220638069119068, 0.5566215826978989, 0.012672106086408326, 0.0, 0.19856537291690143, 0.10948046629883053, 0.09429772625302726, 0.10492020074513651, 0.5122915442940985, 0.4878521416294368], [0.4154788541292638, 0.5085552477366768, 0.38313841411721716, 0.40546035001652425, 0.6358679020664716, 0.14604084035792078, 0.0, 0.6501831862169137, 0.10117424944645306, 0.023091846400179215, 0.17931759084860371, 0.12510166647571977, 0.8425908177248763], [0.4859873504360642, 0.4548998897650739, 0.3990078589839434, 0.4593101169671351, 0.6183407564842326, 0.21271107294549702, 0.0, 0.651211018493526, 0.11199196367822975, 0.02671198221116782, 0.09638600896072039, 0.44336466445717526, 0.5567250069967961], [0.5173265220116787, 0.416371501572967, 0.460902771447204, 0.4297560868510085, 0.5946835555951446, 0.06159858293046262, 0.0, 0.374488981260158, 0.08208188869543587, 0.011861190434747163, 0.10634792182602523, 0.5672083289650874, 0.42906417950136577], [0.4998249648336544, 0.4263312643694995, 0.4294131074967243, 0.4442366923111373, 0.523440209969299, 0.05653931193249666, 0.0, 0.47369579642779425, 0.1540700319307133, 0.13991684813460417, 0.2637325857004414, 0.264709196607401, 0.5006948580591835], [0.5536929351260887, 0.44148593204055586, 0.42612948346575286, 0.5002897827973021, 0.5059717131598763, 0.020292109674663057, 0.0, 0.7645299448699332, 0.08914060658643258, 0.015945782107428086, 0.22084094503801777, 0.7158210173102869, 0.28318004792551416], [0.3927112171583953, 0.5121499156730045, 0.3871165815408256, 0.38309591548897687, 0.6299410496299109, 0.1707987827345705, 0.0, 0.5672528140167444, 0.08921518931724594, 0.01590609460739517, 0.2688046686463201, 3.420385423311072e-05, 0.9360006720142177], [0.4443392219144364, 0.4009232405354001, 0.4252075073897224, 0.39608395383673756, 0.5567969810734474, 0.027410448870971403, 0.0, 0.16691509618023762, 0.13955545790025584, 0.10592735539935547, 0.17500306263255294, 0.0, 0.724356052301631], [0.5090394205433899, 0.4856209562457149, 0.38240404475973083, 0.5006623920153109, 0.5628008327206284, 0.24598217204251288, 0.0, 0.9407977665704408, 0.1140489546184018, 0.012839848972701216, 0.1731247779999318, 0.3502089407586766, 0.46308594276988585], [0.6055980473655983, 0.3844617978872463, 0.4518801845539512, 0.5270154437719061, 0.5040615436881446, 0.009045368385137594, 0.0, 0.6390207181139231, 0.06408749742930289, 0.039933657952823005, 0.16489802524371566, 0.9288775662802417, 0.07112274191428253], [0.5738866048922366, 0.4186626662475822, 0.4028547076455874, 0.5439625438204695, 0.5048118506917832, 0.009919676820019487, 0.0, 0.8480739979491726, 0.08017679015593995, 0.02995039476693146, 0.17597755114441668, 0.798538921651887, 0.20101311135936595], [0.5047757665209491, 0.47004191868778855, 0.46821461872248415, 0.4096540629001449, 0.5142812613934793, 0.02544282808620496, 0.0, 0.5118245599686084, 0.11115149282461441, 0.0026979947748085073, 0.3597604755428314, 0.5180979349701261, 0.4803922180726713], [0.6112363114786946, 0.381672164058778, 0.47535942838485695, 0.5093239448036906, 0.4297421123698417, 0.0053063914820048255, 0.0, 0.5551101957785523, 0.054712356656169296, 0.02871028640601883, 0.0913586146855362, 0.9525707652680174, 0.047411569207795635], [0.5074618669048843, 0.48815650326208326, 0.3827938132545533, 0.4978172054448044, 0.5532326088213593, 0.19726743823764542, 0.0, 0.9433926238790331, 0.12962753940106306, 0.010419190395178989, 0.13165108029677122, 0.5271245009054011, 0.47020475274606355], [0.5214398354093666, 0.46823008452980763, 0.38070063884181493, 0.5138217899289927, 0.5178003092424716, 0.05321626438128043, 0.0, 0.9269408013526372, 0.10250479717970219, 0.021412171925276724, 0.19991575382545312, 0.5870456051180613, 0.4130402803403468], [0.6102157288215986, 0.3790459357678314, 0.434862300036754, 0.5485916657329433, 0.5013418008886639, 0.00413933652248366, 0.0, 0.7028565798718541, 0.06950007465922788, 0.04528588059352065, 0.10640882621835515, 0.9231379333474594, 0.05166388812922704], [0.524159312936452, 0.4394935622462022, 0.4385872747005509, 0.4586828467019515, 0.5621944258586954, 0.09613807522972932, 0.0, 0.5872724496722153, 0.0896869091694984, 0.014575698327125368, 0.158584494866817, 0.5974504716254285, 0.40255491895462664], [0.5585430940823568, 0.42516844682504124, 0.46379166497617474, 0.4717366898849905, 0.5466067626705008, 0.10793590007584182, 0.0, 0.5708757965533306, 0.08380750978624837, 0.029273087009499384, 0.09864384797604363, 0.0008679119940430023, 0.262749120077885], [0.5048718217632848, 0.449820283004401, 0.4418802224911279, 0.43692512639158887, 0.5210822884693738, 0.04416631332692331, 0.0, 0.5397743214722044, 0.13447420440684765, 0.10031936245885095, 0.21935022764380877, 0.2953830994326029, 0.4805146604195085], [0.5529697136792505, 0.41520687967243897, 0.43385348517000083, 0.4920469295761767, 0.5584749853733102, 0.07779801019939853, 0.0, 0.6232310542354493, 0.08354232087770076, 0.021197455212455998, 0.1409307424265465, 0.6778317553322426, 0.28619188718635497], [0.5564349760157932, 0.3958457591703587, 0.44983941699716, 0.47755275567815414, 0.509750511899214, 0.02299326036802849, 0.0, 0.4853917723031097, 0.12441578266038926, 0.17628080263376533, 0.247191234826183, 0.724588090645327, 0.2732885032332819], [0.49519807242072356, 0.4104158594552748, 0.4847574774244774, 0.38766088425427947, 0.5355469959517254, 0.03209310597999469, 0.0, 0.17273659908725708, 0.12704113452661425, 0.08634606387581237, 0.1844669468656832, 0.0, 0.5200052394195017], [0.5306691508930302, 0.4231341454244605, 0.4518520392848887, 0.45173340408572293, 0.5581286137170044, 0.074524225923143, 0.0, 0.49356376990092066, 0.09791215603683409, 0.02263022406235208, 0.17037506846614775, 0.6080295027217706, 0.37622736860953176], [0.43673711737270066, 0.524992608812871, 0.3885422695735333, 0.42145684660090577, 0.6070231917789117, 0.2215384738466737, 0.0, 0.781192635895429, 0.08741506844256386, 0.009526527304662169, 0.12506033139039366, 0.24349057240966038, 0.7564909572495198], [0.5720621976369741, 0.42127597173513276, 0.4174682876145928, 0.5272992844131297, 0.5095754860027939, 0.026108312130987733, 0.0, 0.7902347120366735, 0.08915666813488049, 0.02467692713960304, 0.19839075156288166, 0.7920835241763533, 0.20789406438457178], [0.560802712069927, 0.41306424116873863, 0.4663554264957216, 0.46678610063781323, 0.5053782788374388, 0.014221991051172542, 0.0, 0.5125408468414356, 0.08244248125414241, 0.10521619362270351, 0.2581512544985185, 0.7245734731257418, 0.2555707337042106], [0.6126651631808119, 0.3795360188940815, 0.44196212119843636, 0.5437956943567539, 0.5020743074114794, 0.008223729593645292, 0.0, 0.6857778616210577, 0.07056991954834219, 0.03220472811035159, 0.1476238537993097, 0.9585849723602793, 0.04141575242038168], [0.6101351218182317, 0.3807537461254362, 0.4392175690306829, 0.5439619844338682, 0.5044182709774783, 0.009734393497170249, 0.0, 0.6905232421773461, 0.07077092407145484, 0.0384357118029324, 0.1286954202981243, 0.9129501195716163, 0.05211156839045558], [0.6114700348007124, 0.3847817650579337, 0.5074956715210208, 0.47655816859826095, 0.4180713001240913, 0.012637975346984603, 0.0, 0.4363965703613011, 0.06893751245597235, 0.015623587147256838, 0.1212757990927405, 0.9535654739167381, 0.04643513872804711], [0.5400136691808409, 0.43885943492012575, 0.44099242875146816, 0.4720303259504949, 0.5125875915517623, 0.021778612459129355, 0.0, 0.6376222211925708, 0.10531802511071134, 0.013478985511539154, 0.24824877343560653, 0.6607026588793814, 0.33815446360353385], [0.5603349769194115, 0.39081673462382455, 0.41464021020798, 0.5177677529428917, 0.5078783359396999, 0.019440991018141174, 0.0, 0.6250652563676162, 0.10447263845216734, 0.19185530639137433, 0.2305134808100372, 0.6351175566494265, 0.25745961964854747], [0.4143407853540533, 0.5358152954274082, 0.3874750435796994, 0.40112340308700484, 0.6162620464797444, 0.18856598848085393, 0.0, 0.7382192198541178, 0.09922243747719674, 0.014006587941666872, 0.14890834282619797, 0.13929238252593845, 0.8470063042138807], [0.5042492214992395, 0.4635817361349147, 0.45652353404899226, 0.4238915898464269, 0.5243755061375457, 0.0464885916334281, 0.0, 0.5281441671421865, 0.1242847821334089, 0.04410929573820905, 0.16750555306306, 0.0021781358822414907, 0.483596634685373], [0.5626493733884129, 0.4319612525400486, 0.4094461685806548, 0.5262974814695044, 0.5069783589078314, 0.016269998738347245, 0.0, 0.8309903295645427, 0.09453397728162774, 0.0069741524575620585, 0.15549379642583022, 0.7483965845978238, 0.24669266325492342], [0.4255957324517557, 0.4790619408135769, 0.38604109280014154, 0.4129139538673673, 0.6096204965391498, 0.06906340997463274, 0.0, 0.5611996327184379, 0.08197306823817471, 0.021664639431725457, 0.21835685340415764, 0.19842156727949242, 0.8015837585161809], [0.5107663407251475, 0.4409100831198621, 0.44003240589658865, 0.44294653414609336, 0.5274281055509454, 0.024959974813484838, 0.0, 0.5309339353804926, 0.14085939142994439, 0.01672655925228863, 0.25430211828513466, 0.48686098685772183, 0.45660070616078763]]


In [ ]:
players=np.array(players)
players=players*100

In [ ]:
#clustering the players into clusters
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

players = np.array(players)

silhouette_scores = []

k_range = range(4, 30)  

# Iterate through each number of clusters
for k in k_range:
    # Initialize and fit the k-means model
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(players)
    
    # Predict cluster labels
    labels = kmeans.labels_
    
    # Calculate silhouette score
    silhouette_avg = silhouette_score(players, labels)
    silhouette_scores.append(silhouette_avg)

# Find the index of the maximum silhouette score
optimal_clusters_idx = np.argmax(silhouette_scores)
optimal_clusters = k_range[optimal_clusters_idx]

# Plot silhouette scores
plt.plot(k_range, silhouette_scores, marker='o')
plt.title('Silhouette Score Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.axvline(x=optimal_clusters, color='r', linestyle='--', label=f'Optimal Clusters: {optimal_clusters}')
plt.legend()
plt.show()

print("Optimal Number of Clusters:", optimal_clusters)

# Initialize and fit the k-means model with the optimal number of clusters
kmeans = KMeans(n_clusters=optimal_clusters)
kmeans.fit(players)

# Predict cluster labels
labels = kmeans.labels_



#### Solution to Problem Statement 4
A monte Carlo Simulation for the different Stratergies is as follows:

In [ ]:
import csv
import numpy as np
from sklearn.cluster import KMeans

# Read data from CSV
games = []

with open("input_game.csv", 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # Skip header
    for row in csvreader:
        games.append(row)

# Convert the list of games to a NumPy array
games_array = np.array(games)

# Extract player IDs and actions
player_ids = np.concatenate((games_array[:, 1].astype(int), games_array[:, 2].astype(int)))
player_actions = np.concatenate((games_array[:, 3], games_array[:, 4]))

# Encode player actions as numeric values
action_mapping = {'TRUST': 0, 'CHEAT': 1}
encoded_actions = np.array([action_mapping[action] for action in player_actions])

# Combine player IDs and encoded actions into feature matrix
feature_matrix = np.column_stack((player_ids, encoded_actions))

# Apply K-means clustering
num_clusters = 2  # Adjust this number as needed
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(feature_matrix)
labels = kmeans.labels_

# Print cluster labels
print("Cluster labels for players:")
for player_id, label in zip(player_ids, labels):
    print(f"Player {player_id}: Cluster {label}")